In [104]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import tree
from scipy.stats import iqr
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn.model_selection import GridSearchCV

In [112]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

In [113]:
# trova i parametri ottimali di un classificatore

def optimizeParams(clf, params, X_train, y_train):
    clf_gs = GridSearchCV(clf, params, cv=5)
    clf_gs.fit(X_train, y_train)
    return clf_gs.best_estimator_

In [114]:
# calcola accuracy e f1 score di un classificatore

def metrics(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)*100
    f1 = f1_score(y_test, y_pred, average = 'macro')*100
    return [accuracy, f1]

In [134]:
y = []
X = {'metadata': [], 'statistics' : [], 'timeseries' : []}

path = ""

# calcoliamo le features di ogni timeseries

with open(path + 'ThingspeakEU.meta.csv', 'r', encoding='utf-8') as dati:
    for row in dati:
        riga = row.strip().split(',')
        
        classe = int(riga[8])
        y.append(classe)
        
        valori = np.array(riga[9:]).astype(np.float)
        X['timeseries'].append(valori)
        
        # metadati
        stream_name = riga[1]
        X['metadata'].append(stream_name)
        
        # statistiche
        valori = np.array(riga[9:]).astype(np.float)
        media = np.mean(valori)
        mediana = np.median(valori)
        maxim = np.max(valori)
        minim = np.min(valori)
        std_dev = np.std(valori)
        rms = np.sqrt(np.mean(np.square(valori)))
        quantile = np.quantile(valori, 0.4)
        i_q_r = iqr(valori)
        simmetria = skew(valori)
        curtosi = kurtosis(valori)
        rang = maxim - minim
        
        features = [rang, maxim, std_dev, rms, media, minim, quantile, mediana, curtosi, simmetria, i_q_r] 
        X['statistics'].append(features)
    
X = pd.DataFrame(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 100)

In [121]:
# creiamo i classificatori e la lista di parametri la testare

clfs = []
params = []

decision_tree = RandomForestClassifier()
params_tree = {'n_estimators' : [100, 1000], 'criterion': ['gini', 'entropy'], 'max_depth' : np.arange(5, 25), 'max_features' : np.arange(1, 12)}
clfs.append(decision_tree)
params.append(params_tree)

svm = SVC()
params_svm = {'gamma' : ['scale', 'auto']}
clfs.append(svm)
params.append(params_svm)

knn = KNeighborsClassifier()
params_knn = {'n_neighbors' : np.arange(1, 25)}
clfs.append(knn)
params.append(params_knn)

lg = LogisticRegression()
params_lg = {'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'multi_class': ['ovr', 'auto']}
clfs.append(lg)
params.append(params_lg)

ridge = RidgeClassifier()
params_ridge = {'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
clfs.append(ridge)
params.append(params_ridge)

gnb = GaussianNB()
clfs.append(gnb)

In [122]:
# ottimizziamo i parametri e calcoliamo le metriche dei classificatori ottimizzati

clfs_best = []
metrics_best = []

for clf, param in zip(clfs, params):
    clf_best = optimizeParams(clf, param, list(X_train['statistics']), y_train)
    clfs_best.append(clf_best)
    metrics_best.append(metrics(clf_best, list(X_test['statistics']), y_test))

clfs_best.append(gnb)
gnb.fit(list(X_train['statistics']), y_train)
metrics_best.append(metrics(gnb, list(X_test['statistics']), y_test))

In [124]:
# metriche dei classificatori standard

metrics_standard = []

for clf in clfs:
    clf.fit(list(X_train['statistics']), y_train)
    metrics_standard.append(metrics(clf, list(X_test['statistics']), y_test))

In [125]:
clfs_best

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=11, max_features=6, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=None, n_neighbors=14, p=2,
            weights='uniform'),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr',
           n_jobs=None, penal

In [126]:
from itertools import combinations

In [127]:
# creiamo un voting classifier

best_combination_voting = [0.00, "", ""]
best_ensemble_voting = None

for i in range(2, len(clfs_best)+1):
    combs = combinations(clfs_best, i)
    for comb in combs:
        estimators = []
        for classifier in comb:
            estimators.append((classifier.__class__.__name__, classifier))
        voting = VotingClassifier(estimators)
        voting.fit(list(X_train['statistics']), list(y_train))
        preds = voting.predict(list(X_test['statistics']))
        accuracy = accuracy_score(preds, list(y_test))
        f1 = f1_score(preds, list(y_test), average = 'micro')
    
        if accuracy > best_combination_voting[0]:
            best_combination_voting[0] = accuracy
            best_combination_voting[1] = [estimator[0] for estimator in estimators]
            best_combination_voting[2] = f1
            best_ensemble_voting = ensemble
    
        print("Accuracy score: {:.3f} {}".format(accuracy, [estimator[0] for estimator in estimators]))

print("\nBest voting model is {}, with accuracy of: {:.3f} and f1 score of: {:.3f}".format(best_combination_voting[1], best_combination_voting[0], best_combination_voting[2]))

Accuracy score: 0.630 ['RandomForestClassifier', 'SVC']
Accuracy score: 0.717 ['RandomForestClassifier', 'KNeighborsClassifier']
Accuracy score: 0.650 ['RandomForestClassifier', 'LogisticRegression']
Accuracy score: 0.383 ['RandomForestClassifier', 'RidgeClassifier']
Accuracy score: 0.639 ['RandomForestClassifier', 'GaussianNB']
Accuracy score: 0.622 ['SVC', 'KNeighborsClassifier']
Accuracy score: 0.562 ['SVC', 'LogisticRegression']
Accuracy score: 0.342 ['SVC', 'RidgeClassifier']
Accuracy score: 0.589 ['SVC', 'GaussianNB']
Accuracy score: 0.645 ['KNeighborsClassifier', 'LogisticRegression']
Accuracy score: 0.383 ['KNeighborsClassifier', 'RidgeClassifier']
Accuracy score: 0.636 ['KNeighborsClassifier', 'GaussianNB']
Accuracy score: 0.345 ['LogisticRegression', 'RidgeClassifier']
Accuracy score: 0.567 ['LogisticRegression', 'GaussianNB']
Accuracy score: 0.311 ['RidgeClassifier', 'GaussianNB']
Accuracy score: 0.727 ['RandomForestClassifier', 'SVC', 'KNeighborsClassifier']
Accuracy score:

In [128]:
from mlens.ensemble import SuperLearner

In [129]:
valori_features = []
classi = []

path = ""

# calcoliamo le features di ogni timeseries

with open(path + 'ThingspeakEU.meta.csv', 'r', encoding='utf-8') as dati:
    for row in dati:
        riga = row.strip().split(',')
        classe = int(riga[8])
        classi.append(classe)
        valori = np.array(riga[9:]).astype(np.float)
        media = np.mean(valori)
        mediana = np.median(valori)
        maxim = np.max(valori)
        minim = np.min(valori)
        std_dev = np.std(valori)
        rms = np.sqrt(np.mean(np.square(valori)))
        quantile = np.quantile(valori, 0.4)
        i_q_r = iqr(valori)
        simmetria = skew(valori)
        curtosi = kurtosis(valori)
        rang = maxim - minim
        features = [rang, maxim, std_dev, rms, media, minim, quantile, mediana, curtosi, simmetria, i_q_r] 
        valori_features.append(features)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(valori_features, classi, test_size = 0.3, random_state = 100)

In [131]:
# creiamo uno stacking classifier

names = ['DecisionTree', 'SVM', 'KNeighbors', 'Logistic', 'Ridge', 'GaussianNB']

# funzione che restituisce tutte le possibili combinazioni di classifiers

def zip_stacked_classifiers(*args):
    to_zip = []
    for arg in args:
        combined_items = sum([list(map(list, combinations(arg, i))) for i in range(len(arg) + 1)], [])
        combined_items = filter(lambda x: len(x) > 0, combined_items)
        to_zip.append(combined_items)
    
    return zip(to_zip[0], to_zip[1])

stacked_clf_list = zip_stacked_classifiers(clfs_best, names)

best_combination = [0.00, "", ""]
best_ensemble = None

# troviamo il miglior ensemble, utilizzando come meta classificatore un decision_tree

for clf in stacked_clf_list:
    for meta in clfs:
        ensemble = SuperLearner(random_state = 1000)
        ensemble.add(clf[0])
        ensemble.add_meta(meta)
        ensemble.fit(np.array(X_train), np.array(y_train))
        preds = ensemble.predict(np.array(X_test))
        accuracy = accuracy_score(preds, np.array(y_test))
    
        if accuracy > best_combination[0]:
            best_combination[0] = accuracy
            best_combination[1] = clf[1]
            best_combination[2] = meta.__class__.__name__
            best_ensemble = ensemble
    
        print("Accuracy score: {:.3f} {}".format(accuracy, clf[1]))

print("\nBest stacking model is {}, with meta-learner {} and accuracy of: {:.3f}".format(best_combination[1], best_combination[2], best_combination[0]))

Accuracy score: 0.746 ['DecisionTree']
Accuracy score: 0.754 ['DecisionTree']
Accuracy score: 0.739 ['DecisionTree']
Accuracy score: 0.342 ['DecisionTree']
Accuracy score: 0.181 ['DecisionTree']
Accuracy score: 0.534 ['DecisionTree']
Accuracy score: 0.620 ['SVM']
Accuracy score: 0.623 ['SVM']
Accuracy score: 0.590 ['SVM']
Accuracy score: 0.331 ['SVM']
Accuracy score: 0.331 ['SVM']
Accuracy score: 0.447 ['SVM']
Accuracy score: 0.711 ['KNeighbors']
Accuracy score: 0.710 ['KNeighbors']
Accuracy score: 0.581 ['KNeighbors']
Accuracy score: 0.338 ['KNeighbors']
Accuracy score: 0.177 ['KNeighbors']
Accuracy score: 0.510 ['KNeighbors']
Accuracy score: 0.611 ['Logistic']
Accuracy score: 0.611 ['Logistic']
Accuracy score: 0.595 ['Logistic']
Accuracy score: 0.320 ['Logistic']
Accuracy score: 0.182 ['Logistic']
Accuracy score: 0.460 ['Logistic']
Accuracy score: 0.306 ['Ridge']
Accuracy score: 0.306 ['Ridge']
Accuracy score: 0.242 ['Ridge']
Accuracy score: 0.188 ['Ridge']
Accuracy score: 0.184 ['Ri

Accuracy score: 0.314 ['DecisionTree', 'Logistic', 'Ridge']
Accuracy score: 0.185 ['DecisionTree', 'Logistic', 'Ridge']
Accuracy score: 0.469 ['DecisionTree', 'Logistic', 'Ridge']
Accuracy score: 0.741 ['DecisionTree', 'Logistic', 'GaussianNB']
Accuracy score: 0.738 ['DecisionTree', 'Logistic', 'GaussianNB']
Accuracy score: 0.703 ['DecisionTree', 'Logistic', 'GaussianNB']
Accuracy score: 0.338 ['DecisionTree', 'Logistic', 'GaussianNB']
Accuracy score: 0.177 ['DecisionTree', 'Logistic', 'GaussianNB']
Accuracy score: 0.473 ['DecisionTree', 'Logistic', 'GaussianNB']
Accuracy score: 0.732 ['DecisionTree', 'Ridge', 'GaussianNB']
Accuracy score: 0.735 ['DecisionTree', 'Ridge', 'GaussianNB']
Accuracy score: 0.675 ['DecisionTree', 'Ridge', 'GaussianNB']
Accuracy score: 0.441 ['DecisionTree', 'Ridge', 'GaussianNB']
Accuracy score: 0.176 ['DecisionTree', 'Ridge', 'GaussianNB']
Accuracy score: 0.471 ['DecisionTree', 'Ridge', 'GaussianNB']
Accuracy score: 0.694 ['SVM', 'KNeighbors', 'Logistic']
Ac

Accuracy score: 0.727 ['DecisionTree', 'Logistic', 'Ridge', 'GaussianNB']
Accuracy score: 0.733 ['DecisionTree', 'Logistic', 'Ridge', 'GaussianNB']
Accuracy score: 0.703 ['DecisionTree', 'Logistic', 'Ridge', 'GaussianNB']
Accuracy score: 0.432 ['DecisionTree', 'Logistic', 'Ridge', 'GaussianNB']
Accuracy score: 0.177 ['DecisionTree', 'Logistic', 'Ridge', 'GaussianNB']
Accuracy score: 0.499 ['DecisionTree', 'Logistic', 'Ridge', 'GaussianNB']
Accuracy score: 0.692 ['SVM', 'KNeighbors', 'Logistic', 'Ridge']
Accuracy score: 0.689 ['SVM', 'KNeighbors', 'Logistic', 'Ridge']
Accuracy score: 0.680 ['SVM', 'KNeighbors', 'Logistic', 'Ridge']
Accuracy score: 0.429 ['SVM', 'KNeighbors', 'Logistic', 'Ridge']
Accuracy score: 0.353 ['SVM', 'KNeighbors', 'Logistic', 'Ridge']
Accuracy score: 0.388 ['SVM', 'KNeighbors', 'Logistic', 'Ridge']
Accuracy score: 0.691 ['SVM', 'KNeighbors', 'Logistic', 'GaussianNB']
Accuracy score: 0.694 ['SVM', 'KNeighbors', 'Logistic', 'GaussianNB']
Accuracy score: 0.673 ['SV

In [132]:
from ipynb.fs.defs.Ensemble_Framework import EnsembleFramework

In [135]:
clf = EnsembleFramework(criterion = 'topk', layers = [{'type' : 'BOS', 'name' : 'RandomForestClassifier(bootstrap=True, class_weight=None, criterion=\'gini\',max_depth=11, max_features=6, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,oob_score=False, random_state=None, verbose=0,warm_start=False)'},  {'type' : 'BOS', 'name' : 'SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=\'ovr\', degree=3, gamma=\'auto\', kernel=\'rbf\',max_iter=-1, probability=True, random_state=None, shrinking=True,tol=0.001, verbose=False)'}, {'type' : 'BOS', 'name' : 'KNeighborsClassifier(algorithm=\'auto\', leaf_size=30, metric=\'minkowski\',metric_params=None, n_jobs=None, n_neighbors=14, p=2,weights=\'uniform\')'}], params = {'k' : [7, 4, 1]})
y_pred = clf.run(X_train, y_train, X_test)

In [136]:
clf.accuracy(y_pred, y_test)

70.95761381475667

In [137]:
clf = EnsembleFramework(criterion = 'topk', layers = [{'type' : 'BOS', 'name' : 'RandomForestClassifier(bootstrap=True, class_weight=None, criterion=\'gini\',max_depth=11, max_features=6, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,oob_score=False, random_state=None, verbose=0,warm_start=False)'},  {'type' : 'BOS', 'name' : 'SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=\'ovr\', degree=3, gamma=\'auto\', kernel=\'rbf\',max_iter=-1, probability=True, random_state=None, shrinking=True,tol=0.001, verbose=False)'}, {'type' : 'BOS', 'name' : 'RandomForestClassifier(bootstrap=True, class_weight=None, criterion=\'gini\',max_depth=11, max_features=6, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,oob_score=False, random_state=None, verbose=0,warm_start=False)'}], params = {'k' : [7, 4, 1]})
y_pred = clf.run(X_train, y_train, X_test)

In [138]:
clf.accuracy(y_pred, y_test)

72.8414442700157

In [139]:
clf = EnsembleFramework(criterion = 'sof', layers = [{'type' : 'BOS', 'name' : 'RandomForestClassifier(bootstrap=True, class_weight=None, criterion=\'gini\',max_depth=11, max_features=6, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,oob_score=False, random_state=None, verbose=0,warm_start=False)'},  {'type' : 'BOS', 'name' : 'SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=\'ovr\', degree=3, gamma=\'auto\', kernel=\'rbf\',max_iter=-1, probability=True, random_state=None, shrinking=True,tol=0.001, verbose=False)'}, {'type' : 'BOS', 'name' : 'KNeighborsClassifier(algorithm=\'auto\', leaf_size=30, metric=\'minkowski\',metric_params=None, n_jobs=None, n_neighbors=14, p=2,weights=\'uniform\')'}])
y_pred = clf.run(X_train, y_train, X_test)

In [140]:
clf.accuracy(y_pred, y_test)

75.35321821036108

In [141]:
clf = EnsembleFramework(criterion = 'sof', layers = [{'type' : 'BOS', 'name' : 'RandomForestClassifier(bootstrap=True, class_weight=None, criterion=\'gini\',max_depth=11, max_features=6, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,oob_score=False, random_state=None, verbose=0,warm_start=False)'},  {'type' : 'BOS', 'name' : 'SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,decision_function_shape=\'ovr\', degree=3, gamma=\'auto\', kernel=\'rbf\',max_iter=-1, probability=True, random_state=None, shrinking=True,tol=0.001, verbose=False)'}, {'type' : 'BOS', 'name' : 'RandomForestClassifier(bootstrap=True, class_weight=None, criterion=\'gini\',max_depth=11, max_features=6, max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,oob_score=False, random_state=None, verbose=0,warm_start=False)'}])
y_pred = clf.run(X_train, y_train, X_test)

In [142]:
clf.accuracy(y_pred, y_test)

75.35321821036108

In [146]:
stacked_clf_list = zip_stacked_classifiers(clfs_best, names)

In [144]:
best_combination = [0.00, ""]
best_ensemble = None

# troviamo il miglior ensemble, utilizzando come meta classificatore un decision_tree

for clf_list in stacked_clf_list:
    l = []
    for classifier in clf_list[0]:
        dd = {'type' : 'BOS', 'name' : str(classifier)}
        l.append(dd)
    ensemble = EnsembleFramework(criterion = 'sof', layers = l)
    y_pred = clf.run(X_train, y_train, X_test)    
    accuracy = clf.accuracy(y_pred, y_test)
    
    if accuracy > best_combination[0]:
        best_combination[0] = accuracy
        best_combination[1] = clf_list[1]
        best_ensemble = clf
    
    print("Accuracy score: {:.3f} {}".format(accuracy, clf_list[1]))

print("\nBest stacking model is {}, with accuracy of: {:.3f}".format(best_combination[1], best_combination[0]))

Accuracy score: 74.882 ['DecisionTree']
Accuracy score: 75.196 ['SVM']
Accuracy score: 75.039 ['KNeighbors']
Accuracy score: 74.882 ['Logistic']
Accuracy score: 75.353 ['Ridge']
Accuracy score: 75.667 ['GaussianNB']
Accuracy score: 74.568 ['DecisionTree', 'SVM']
Accuracy score: 75.667 ['DecisionTree', 'KNeighbors']
Accuracy score: 75.039 ['DecisionTree', 'Logistic']
Accuracy score: 75.353 ['DecisionTree', 'Ridge']
Accuracy score: 75.039 ['DecisionTree', 'GaussianNB']
Accuracy score: 75.196 ['SVM', 'KNeighbors']
Accuracy score: 75.353 ['SVM', 'Logistic']
Accuracy score: 74.725 ['SVM', 'Ridge']
Accuracy score: 75.353 ['SVM', 'GaussianNB']
Accuracy score: 74.725 ['KNeighbors', 'Logistic']
Accuracy score: 74.725 ['KNeighbors', 'Ridge']
Accuracy score: 74.411 ['KNeighbors', 'GaussianNB']
Accuracy score: 75.510 ['Logistic', 'Ridge']
Accuracy score: 75.196 ['Logistic', 'GaussianNB']
Accuracy score: 74.725 ['Ridge', 'GaussianNB']
Accuracy score: 75.510 ['DecisionTree', 'SVM', 'KNeighbors']
Acc

In [147]:
best_combination = [0.00, ""]
best_ensemble = None

# troviamo il miglior ensemble, utilizzando come meta classificatore un decision_tree

for clf_list in stacked_clf_list:
    l = []
    for classifier in clf_list[0]:
        dd = {'type' : 'BOS', 'name' : str(classifier)}
        l.append(dd)
    if (len(clf_list[0]) == 1):
        para = {'k' : [1]}
    if (len(clf_list[0]) == 2):
        para = {'k' : [4, 1]}
    if (len(clf_list[0]) == 3):
        para = {'k' : [5, 3, 1]}
    if (len(clf_list[0]) == 4):
        para = {'k' : [7, 5, 3, 1]}
    if (len(clf_list[0]) == 5):
        para = {'k' : [11, 8, 6, 4, 1]}
    if (len(clf_list[0]) == 6):
        para = {'k' : [15, 12, 9, 6, 4, 1]}
    ensemble = EnsembleFramework(criterion = 'topk', layers = l, params = para)
    y_pred = clf.run(X_train, y_train, X_test)    
    accuracy = clf.accuracy(y_pred, y_test)
    
    if accuracy > best_combination[0]:
        best_combination[0] = accuracy
        best_combination[1] = clf_list[1]
        best_ensemble = clf
    
    print("Accuracy score: {:.3f} {}".format(accuracy, clf_list[1]))

print("\nBest stacking model is {}, with accuracy of: {:.3f}".format(best_combination[1], best_combination[0]))

Accuracy score: 75.353 ['DecisionTree']
Accuracy score: 75.196 ['SVM']
Accuracy score: 74.882 ['KNeighbors']
Accuracy score: 75.039 ['Logistic']
Accuracy score: 75.196 ['Ridge']
Accuracy score: 75.039 ['GaussianNB']
Accuracy score: 75.196 ['DecisionTree', 'SVM']
Accuracy score: 75.510 ['DecisionTree', 'KNeighbors']
Accuracy score: 75.039 ['DecisionTree', 'Logistic']
Accuracy score: 75.039 ['DecisionTree', 'Ridge']
Accuracy score: 74.882 ['DecisionTree', 'GaussianNB']
Accuracy score: 75.039 ['SVM', 'KNeighbors']
Accuracy score: 75.510 ['SVM', 'Logistic']
Accuracy score: 75.981 ['SVM', 'Ridge']
Accuracy score: 75.510 ['SVM', 'GaussianNB']
Accuracy score: 75.196 ['KNeighbors', 'Logistic']
Accuracy score: 75.981 ['KNeighbors', 'Ridge']
Accuracy score: 74.725 ['KNeighbors', 'GaussianNB']
Accuracy score: 75.510 ['Logistic', 'Ridge']
Accuracy score: 75.667 ['Logistic', 'GaussianNB']
Accuracy score: 74.882 ['Ridge', 'GaussianNB']
Accuracy score: 75.039 ['DecisionTree', 'SVM', 'KNeighbors']
Acc